In [2]:
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
#import seaborn as sns
import torch
from IPython.display import Image
import torch

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [3]:
sns.set_style('darkgrid')

NameError: name 'sns' is not defined

In [4]:
DATA_DIR = '../data/FairFace'

# Prepare samples for unbiased FID statistic calculation

## Single-attribute

combine all data across splits to maximize number of samples

In [5]:
splits = ['val', 'train']

In [6]:
data = []
labels = []
for split in splits:
    d = torch.load(os.path.join(DATA_DIR, '{}_FairFace_64x64.pt'.format(split)))
    l = torch.load(os.path.join(DATA_DIR, '{}_labels_FairFace_64x64.pt'.format(split)))
    data.append(d)
    labels.append(l)

In [7]:
data = torch.cat(data)
labels = torch.cat(labels)

In [8]:
data.shape, labels.shape

(torch.Size([32401, 3, 64, 64]), torch.Size([32401]))

In [9]:
# get minimum value of class
val, freq = np.unique(labels.data.numpy(), return_counts=True)
min_value = min(freq)
print(val, freq)
print(min(freq))

[0. 1.] [18612 13789]
13789


In [10]:
samples = []
ys = []
for i in range(len(val)):
    idx = np.where(labels.data.numpy() == i)[0][0:min_value]
    samples.append(data[idx])
    ys.append(labels[idx])

In [11]:
len(samples)

2

In [12]:
samples[0].shape, ys[0].shape, samples[1].shape, ys[1].shape

(torch.Size([13789, 3, 64, 64]),
 torch.Size([13789]),
 torch.Size([13789, 3, 64, 64]),
 torch.Size([13789]))

In [13]:
samples = torch.cat(samples)
samples = samples.numpy()
samples.shape

(27578, 3, 64, 64)

In [14]:
# now we should convert to numpy
samples = np.array(samples)

In [15]:
samples.shape

(27578, 3, 64, 64)

In [16]:
np.savez('../fid_stats/FairFace/unbiased_all_race_samples.npz', **{'x':samples})